In [1]:
import sys
import pdb
import gym
import math
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from tensorboardX import SummaryWriter
from BayesianNetwork import BayesianNetwork
from BayesianQNetwork import BQN_learn

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def test_agent(agent):
    
    count = 0    
    test_episode_rew = 0
    test_return = []
    
    done = False
    agent.dbqn.eval()
    
    obs = env.reset()
    act = agent.act(obs, use_sample=True, num_sample=4)
    
    while count <= 99:
        if done:
            test_return.append(test_episode_rew)
            test_episode_rew = 0
            count = count + 1
            
            obs = env.reset()
            act = agent.act(obs, use_sample=True, num_sample=4)
        
        obs1, rew, done, _ = env.step(action_modify(act))        
        act = agent.act(obs1, use_sample=True, num_sample=0)                
        test_episode_rew = test_episode_rew + rew        
    
    agent.dbqn.train()
    return np.mean(np.array(test_return))

In [4]:
lr = 1e-3
batch_size = 64
gamma = 0.9
steps = 200000
train_start = 1000
buffer_size = 100000
features_list = [2, 32, 5]
envt = "Pendulum"
env = gym.make('Pendulum-v0')
        
def action_modify(act):
    if envt == 'Chain':
        act = 2*act - 1
    if envt == 'Pendulum':
        act = [int(-2 + act)]
    return act

def obs_modify(obs):
    if envt == 'Pendulum':
        obs = np.array([np.arctan2(obs[0], obs[1]), obs[2]])
    return obs    

In [ ]:
runs = 5
run_result = []

for run in range(runs):
                    
    writer = SummaryWriter('results/Pendulum/run' + str(run))
    dbqn = BayesianNetwork(features_list, 4, batch_size, steps).to(DEVICE)    
    target_dbqn = BayesianNetwork(features_list, 4, batch_size, steps).to(DEVICE)
    agent = BQN_learn(dbqn, target_dbqn, gamma, lr, batch_size, buffer_size, writer, train_start)
    
    done = False    
    episode_rew = 0
    episode_count = 0
    res = []

#     pdb.set_trace()
    obs = env.reset()
    obs = obs_modify(obs)
    act = agent.reset(obs)         
    
    while agent.t <= steps and episode_count < 1000:
        
        if done:
            print("Episode " + str(episode_count) + " with reward = " + str(episode_rew))  
            writer.add_scalar('data/reward', episode_rew, episode_count)            
            res.append(episode_rew)
            
            episode_rew = 0
            episode_count = episode_count + 1                            
                                    
            if episode_count%25 == 0:
                test_result = test_agent(agent)
                print("Test Result = " + str(test_result))
                writer.add_scalar('data/test_reward', test_result, episode_count)
                
            obs = env.reset()
            obs = obs_modify(obs)
            act = agent.reset(obs)   

        obs1, rew, done, _ = env.step(action_modify(act))
        obs1 = obs_modify(obs1)
        act = agent.step(obs, act, rew, obs1, done)
        obs = obs1
        episode_rew = episode_rew + rew          
                
    writer.close()
        
    run_result.append(res)

Episode 0 with reward = -781.5248561676339
Episode 1 with reward = -1153.6123760291443
Episode 2 with reward = -1729.2588037159212
Episode 3 with reward = -1325.5668481687762
Episode 4 with reward = -1240.250693511807
Episode 5 with reward = -1460.7400156564509
Episode 6 with reward = -1012.8984905328812
Episode 7 with reward = -973.4447589078619
Episode 8 with reward = -846.8681109277429
